In [1]:
import cv2
import os
import re
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import csv
import pandas as pd
import keras
from keras import models,Model
from keras.layers import Dense,GaussianNoise, Dropout,Input
%matplotlib inline
from skimage.feature import hog

/home/jeet/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jeet/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataSet = []
person = []
horiSet = []

count = 0
folder = "/home/jeet/ML face images train"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
#         print(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200,200))
        fd, img = hog(img,visualise=True)
        img = np.ravel(img)
        if img is not None:
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person.append(numbers[0])
            horiSet.append(img.T)
            dataSet.append(img)
            count = count + 1
           
        
dataSet_test = []
person_test = []
horiSet_test = []

folder = "/home/jeet/ML face images test"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        fd, img = hog(img, visualise=True)
        img = np.ravel(img)
        if img is not None:
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person_test.append(numbers[0])
            horiSet_test.append(img.T)
            dataSet_test.append(img)
            count = count + 1

/home/jeet/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [3]:
#Getting data in desired format and mapping the lables
dataSet = np.array(dataSet)
train = pd.DataFrame(dataSet)

person = np.array(person)
train['id'] = person.transpose()

unique_person = train['id']
unique_person = np.array(unique_person)
unique_person = np.unique(unique_person)
key = range(len(unique_person))

roll_dict = dict( zip(unique_person, key))

mapped_roll = np.vectorize(roll_dict.get)(person)
train['id'] = mapped_roll.transpose()

y = train['id'].values.astype('int64')
images = train.drop(['id'], axis=1, inplace=False)
x = (images.values).astype('uint8')

X_train = x
Y_train = y

dataSet_test = np.array(dataSet_test)
test = pd.DataFrame(dataSet_test)

person_test = np.array(person_test)
test['id'] = person_test.transpose()
print(test)

mapped_roll_test = np.vectorize(roll_dict.get)(person_test)
test['id'] = mapped_roll_test.transpose()

y_test = test['id'].values.astype('int64')
images = test.drop(['id'], axis=1, inplace=False)
x_test = (images.values).astype('uint8')

X_test = x_test
Y_test = y_test

       0    1    2    3    4    5    6    7    8    9    ...      39991  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
9    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
10   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
11   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...        0.0   
12   0.0  0.0  0.0  0.0  

In [4]:
# img1 = cv2.imread('/home/jeet/ML face images test/201501067O_disgust.jpg')
# img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
# img1 = cv2.resize(img1, (200,200))
# fd, img = hog(img1,visualise=True)
# img = np.ravel(img)

min_arr = []
distance = []

for test_data in X_test:
    distance = []
#     print("\n In distance: ",distance)
    for train_data in X_train:
        dist = np.linalg.norm(test_data-train_data)
        distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
    min_arr.append(distance.index(np.min(distance)))
#     min_arr.append(np.min(distance))

In [5]:
print(len(min_arr))

140


In [7]:
print(Y_test)

[29 32  4 14 34 35 25 29  6 18 25 19 21 24 28 33 10  2  7  9 23 31 15 12
  6 21 20  8 26 34  4  4 22  1 30 10  1 22 15 16 17 21 15 13 11 16 27 33
 31 29  8 17 19 16 27 11 24 18 25 13  7 11 24 30 26 25  3 24 11  1 20 12
 19 13  2 16 35 32 29  1  3 28  3 23 21  2 22  0 32 22  3  7  7  9 10 32
 12 12 14  0 30 20 27  8 28 28  0  2  4 34 35 31 15 23 17  9 33 19 26 33
 23 18 27 14 10  0 20 31  6 17 34  8 35 26 18  6 30 14  9 13]


In [8]:
Y_train

array([30, 20, 34, 31, 35, 22, 20, 26, 10, 20,  3, 21, 31, 21, 25,  1, 15,
        6,  0, 16, 15, 17, 24, 16, 14,  1,  2, 13, 26,  7, 15, 12, 30,  5,
        8, 11,  9, 12, 21,  8, 35, 15, 18, 10, 29, 23, 34, 15, 13,  4, 32,
       22,  9,  6,  9, 19,  0,  0, 31, 18,  9,  3, 14,  1, 11, 33, 11, 33,
       14, 25, 32,  8, 13, 12, 19,  9, 23, 34,  2,  1, 21, 17, 29, 22,  4,
        9, 29, 13, 15, 16,  8, 18, 33, 21,  9,  7, 28, 13, 12,  0, 31, 26,
       24,  1, 35,  8,  0,  7,  0, 29,  4,  6, 35,  2, 35, 29, 29,  3, 34,
       27,  4, 19, 24,  8, 20, 31, 28, 23,  6, 28,  6, 26, 33, 17, 33, 31,
       12, 10,  3,  0,  4, 18, 34,  8, 26, 35,  2,  0, 15, 26, 24, 13, 15,
       10, 11, 10, 28, 33,  7, 23, 26, 27, 26,  6, 23,  4, 27, 19, 24, 26,
       22,  5, 29,  4, 12, 32, 34,  1, 32, 24, 30, 12,  2, 15, 23,  7,  8,
       21, 14, 22, 17, 25, 11, 24, 21,  8,  4,  8, 26,  1,  6, 16,  4, 23,
       28, 20, 17, 31, 21,  6, 25, 25,  1,  1, 26, 10, 21, 12, 33, 19, 13,
       10, 32,  4, 29,  5

In [9]:
img1 = cv2.imread('/home/jeet/ML face images test/201501067O_disgust.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
img1 = cv2.resize(img1, (200,200))
fd, img = hog(img1,visualise=True)
img = np.ravel(img)

min_arr = []
distance = []

for train_data in X_train:
    dist = np.linalg.norm(img-train_data)
    distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
    min_arr.append(distance.index(np.min(distance)))
#     min_arr.append(np.min(distance))

/home/jeet/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [10]:
print(distance.index(np.min(distance)))

389


In [11]:
Y_train[389]

25

In [12]:
Y_test

array([29, 32,  4, 14, 34, 35, 25, 29,  6, 18, 25, 19, 21, 24, 28, 33, 10,
        2,  7,  9, 23, 31, 15, 12,  6, 21, 20,  8, 26, 34,  4,  4, 22,  1,
       30, 10,  1, 22, 15, 16, 17, 21, 15, 13, 11, 16, 27, 33, 31, 29,  8,
       17, 19, 16, 27, 11, 24, 18, 25, 13,  7, 11, 24, 30, 26, 25,  3, 24,
       11,  1, 20, 12, 19, 13,  2, 16, 35, 32, 29,  1,  3, 28,  3, 23, 21,
        2, 22,  0, 32, 22,  3,  7,  7,  9, 10, 32, 12, 12, 14,  0, 30, 20,
       27,  8, 28, 28,  0,  2,  4, 34, 35, 31, 15, 23, 17,  9, 33, 19, 26,
       33, 23, 18, 27, 14, 10,  0, 20, 31,  6, 17, 34,  8, 35, 26, 18,  6,
       30, 14,  9, 13])

In [25]:
folder = "/home/jeet/ML face images test"

count=0
for filename in os.listdir(folder):
        distance=[]
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        fd, img = hog(img, visualise=True)
        img = np.ravel(img)
        print("file : ",filename)
        
        for train_data in X_train:
            dist = np.linalg.norm(img-train_data)
            distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
            min_arr.append(distance.index(np.min(distance)))
        dis = distance.index(np.min(distance))
        print(Y_train[dis])
        
        for roll in roll_dict:
            if str(roll) in filename:
                print(roll)
                print(roll_dict[roll])
                
                if roll_dict[roll] == Y_train[dis]:
                    count = count+1
                    
print(count)
    

/home/jeet/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


file :  201501086O_disgust.jpg
29
201501086
29
file :  201501095O_fear.jpg
29
201501095
32
file :  1849004O_fear.JPG
4
1849004
4
file :  201501028O_disgust.JPG
29
201501028
14
file :  201501104O_fear.jpg
6
201501104
34
file :  201501109O_fear.jpg
11
201501109
35
file :  201501067_disgust.jpg
25
201501067
25
file :  201501086O_fear.jpg
29
201501086
29
file :  201501004_disgust.JPG
6
201501004
6
file :  201501038_fear.jpg
18
201501038
18
file :  201501067O_disgust.jpg
25
201501067
25
file :  201501039O_fear.jpg
19
201501039
19
file :  201501053_fear.jpg
21
201501053
21
file :  201501060_fear.jpg
16
201501060
24
file :  201501077_fear.jpg
28
201501077
28
file :  201501097O_fear.jpg
29
201501097
33
file :  201501011_disgust.jpg
2
201501011
10
file :  1744003O_disgust.jpg
2
1744003
2
file :  201501007O_disgust.jpg
16
201501007
7
file :  201501009_fear.jpg
18
201501009
9
file :  201501055_fear.jpg
23
201501055
23
file :  201501091O_disgust.jpg
31
201501091
31
file :  201501031_disgust.jpg
15

In [21]:
roll_dict


{1744001: 0,
 1744002: 1,
 1744003: 2,
 1744004: 3,
 1849004: 4,
 20180119: 5,
 201501004: 6,
 201501007: 7,
 201501008: 8,
 201501009: 9,
 201501011: 10,
 201501012: 11,
 201501021: 12,
 201501025: 13,
 201501028: 14,
 201501031: 15,
 201501032: 16,
 201501034: 17,
 201501038: 18,
 201501039: 19,
 201501051: 20,
 201501053: 21,
 201501054: 22,
 201501055: 23,
 201501060: 24,
 201501067: 25,
 201501070: 26,
 201501071: 27,
 201501077: 28,
 201501086: 29,
 201501088: 30,
 201501091: 31,
 201501095: 32,
 201501097: 33,
 201501104: 34,
 201501109: 35}